In [1]:
import simpy
import random
import statistics

Line up for tickets

In [2]:
wait_times = []

In [3]:
# Environment
class Ticket_Counts(object):
    def __init__(self, env, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.cashier = simpy.Resource(env, num_cashiers)
        self.servers = simpy.Resource(env, num_servers)
        self.ushers = simpy.Resource(env, num_ushers)
        
    
        
    def purchase_ticket(self, buyers):
        yield self.env.timeout(random.randint(2, 4))
        # Set ticket buying time
        
    def check_ticket(self, buyers):
        yield self.env.timeout(random.randint(0, 1))
        # Set ticket buying time
        
    def other_sell(self, buyers):
        yield self.env.timeout(random.randint(5, 10))
        # Set ticket buying time
        
        

In [4]:
    # Process 
    def go_to_sites(env, buyer, counts):
        # buyer arrives at the ticket counts
        arrival_time = env.now

        # buy tickets
        with counts.cashier.request() as request:
            yield request
            yield env.process(counts.purchase_ticket(buyer))
            print('after buy tickets: %d' %int(env.now))


        # check tickets
        with counts.ushers.request() as request:
            yield request
            yield env.process(counts.check_ticket(buyer))
            print('after check tickets: %d'%int(env.now))
            print('----------------------------------')

        # other random behaviors
        if random.choice([True, False]):
            with counts.servers.request() as request:
                yield request
                yield env.process(counts.other_sell(buyer))
                print('after other random behaviors: %d'%int(env.now))
                print('----------------------------------')


        # buyer heads into the site
        wait_times.append(env.now - arrival_time)
            

In [5]:
    # Run
    def run(env, num_cashiers, num_servers, num_ushers):
        ticket_counts = Ticket_Counts(env, num_cashiers, num_servers, num_ushers)
    
        for buyer in range(3):
            env.process(go_to_sites(env, buyer, ticket_counts))

        while True:
            yield env.timeout(0.50)  
            # Wait a bit before generating a new person
            buyer += 1
            env.process(go_to_sites(env, buyer, ticket_counts))

In [6]:
    # Calculation
    def get_average_wait_time(wait_times):
        average_wait = statistics.mean(wait_times)

        minutes, frac_minutes = divmod(average_wait, 1)
        seconds = frac_minutes * 60
        return round(minutes), round(seconds)

In [7]:
    # Define number of workers
    def get_user_input():
        num_cashiers = input("Number of cashiers working: ")
        num_servers = input("Number of servers working: ")
        num_ushers = input("Number of ushers working: ")
        params = [num_cashiers, num_servers, num_ushers]

        # Check input is valid
        if all(str(i).isdigit() for i in params):  
            params = [int(x) for x in params]
        else:
            print("Could not parse input. The simulation will use default values:","\n1 cashier, 1 server, 1 usher.",)
            params = [1, 1, 1]
        return params

In [8]:
    # Define function
    def main():
        # Setup
        random.seed(5)
        num_cashiers, num_servers, num_ushers = get_user_input()

        # Run the simulation
        env = simpy.Environment()
        env.process(run(env, num_cashiers, num_servers, num_ushers))
        env.run(until=100)

        # View the results
        mins, secs = get_average_wait_time(wait_times)
        print("Running simulation...",f"\nThe average wait time is {mins} minutes and {secs} seconds.",)

In [9]:
 if __name__ == '__main__':
    main()

Number of cashiers working: 5
Number of servers working: 2
Number of ushers working: 10
after buy tickets: 3
after check tickets: 3
----------------------------------
after buy tickets: 3
after check tickets: 3
----------------------------------
after buy tickets: 4
after buy tickets: 4
after check tickets: 4
----------------------------------
after buy tickets: 5
after check tickets: 5
----------------------------------
after check tickets: 5
----------------------------------
after buy tickets: 5
after buy tickets: 6
after check tickets: 6
----------------------------------
after check tickets: 6
----------------------------------
after buy tickets: 7
after check tickets: 7
----------------------------------
after buy tickets: 8
after check tickets: 8
----------------------------------
after buy tickets: 8
after check tickets: 8
----------------------------------
after buy tickets: 9
after buy tickets: 9
after check tickets: 9
----------------------------------
after check tickets: 9